In [1]:
import os
import re
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# check if the API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("API key is not set")

In [4]:
# initialize the OpenAI client
client = OpenAI()

In [5]:
def extract_pattern(text, pattern):
    """
    Extracts a pattern from the given text.

    Args:
        text (str): The text to search within.
        pattern (str): The regex pattern to search for.

    Returns:
        str: The extracted pattern or None if not found.
    """
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

In [6]:
def run_openai_agent(query):
    print("### Run OpenAI agent")
    print(f"User query: {query}")
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",  
            messages=[
                {"role": "system", "content": "You are a Python coding assistant. Return a Python function for the given query, followed by a test case using assertion. Format your response with '## Python function' and '## Test case' headers."},
                {"role": "user", "content": query}
            ]
        )
        result = completion.choices[0].message.content
        print("Raw response from OpenAI:")
        print(result)
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None

In [7]:
def extract_pattern(text, pattern):
    """
    Extracts a pattern from the given text.

    Args:
        text (str): The text to search within.
        pattern (str): The regex pattern to search for.

    Returns:
        str: The extracted pattern or None if not found.
    """
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

In [8]:
def extract_code(result):
    retry = False
    print("### Extracting Python function")
    python_function = extract_pattern(result, r'## Python function(.*?)## Test case')
    if not python_function:
        retry = True
        print("Python function failed to generate or wrong output format. Setting retry to True.")
        print("Attempting to extract entire response as Python function.")
        python_function = result
    else:
        # Remove markdown code block delimiters and any leading/trailing whitespace
        python_function = re.sub(r'```python\s*', '', python_function)
        python_function = re.sub(r'\s*```\s*', '', python_function)
        python_function = python_function.strip()

    print("### Extracting test case")
    test_function = extract_pattern(result, r'## Test case(.*?)$')
    if not test_function:
        retry = True
        print("Test function failed to generate or wrong output format. Setting retry to True.")
        print("Attempting to create a simple test case.")
        test_function = "assert True  # Placeholder test"
    else:
        # Remove markdown code block delimiters and any leading/trailing whitespace
        test_function = re.sub(r'```python\s*', '', test_function)
        test_function = re.sub(r'\s*```\s*', '', test_function)
        test_function = test_function.strip()

    print("Extracted Python function:")
    print(python_function)
    print("Extracted test case:")
    print(test_function)

    return python_function, test_function, retry

In [9]:
import traceback

def check_code(python_function, test_function):
    retry = False
    try:
        print("Executing Python function:")
        print(python_function)
        exec(python_function)
        print("Code executed successfully.")
        try:
            print("Executing test case:")
            print(test_function)
            exec(test_function)
            print("Code passed test case.")
        except Exception as e:
            print(f"Test failed with error: {str(e)}")
            print(traceback.format_exc())
            retry = True
            print("Setting retry to True")
    except Exception as e:
        print(f"Code failed with error: {str(e)}")
        print(traceback.format_exc())
        retry = True
        print("Setting retry to True")
    return retry

In [10]:
def run_workflow(query):
    print("### ENTER WORKFLOW")
    i = 0
    max_retries = 3
    retry = True
    while i < max_retries and retry:
        print(f"TRY # {i}")
        i += 1
        result = run_openai_agent(query)
        python_function, test_function, retry = extract_code(result)
        retry = check_code(python_function, test_function)

    if not retry:
        print(f"Validated Python function: ```{python_function}```")
    print("### EXIT WORKFLOW")

In [11]:
run_workflow("How can I remove duplicates from a list")

### ENTER WORKFLOW
TRY # 0
### Run OpenAI agent
User query: How can I remove duplicates from a list
Raw response from OpenAI:
## Python function

```python
def remove_duplicates(input_list):
    """
    Removes duplicates from the input list and returns a list with unique elements only.
    :param input_list: List from which duplicates need to be removed
    :return: List with no duplicates, maintaining the original order
    """
    seen = set()
    unique_list = []
    for item in input_list:
        if item not in seen:
            seen.add(item)
            unique_list.append(item)
    return unique_list
```

## Test case

```python
# Test case
test_list = [1, 2, 3, 1, 2, 4, 5]
assert remove_duplicates(test_list) == [1, 2, 3, 4, 5]

print("Test passed!")
```
### Extracting Python function
### Extracting test case
Extracted Python function:
def remove_duplicates(input_list):
    """
    Removes duplicates from the input list and returns a list with unique elements only.
    :param i

In [12]:
run_workflow("How can i find the 2nd largest number in a list")

### ENTER WORKFLOW
TRY # 0
### Run OpenAI agent
User query: How can i find the 2nd largest number in a list
Raw response from OpenAI:
## Python function
```python
def second_largest(numbers):
    if len(numbers) < 2:
        raise ValueError("List must contain at least two numbers.")
    
    first = second = float('-inf')
    for number in numbers:
        if number > first:
            second = first
            first = number
        elif first > number > second:
            second = number
    
    if second == float('-inf'):
        raise ValueError("No second largest element found; the list may contain duplicates of a single number.")
    
    return second
```

## Test case
```python
# Test case to verify the function
numbers = [10, 5, 8, 12, 15, 7, 18]
assert second_largest(numbers) == 15, "Test case failed"
print("Test case passed")
```
### Extracting Python function
### Extracting test case
Extracted Python function:
def second_largest(numbers):
    if len(numbers) < 2:
     

In [13]:
run_workflow("How can i calculate somebodies 1 rep max in weightlifting")

### ENTER WORKFLOW
TRY # 0
### Run OpenAI agent
User query: How can i calculate somebodies 1 rep max in weightlifting
Raw response from OpenAI:
## Python function

To calculate someone's one-repetition maximum (1RM) in weightlifting, we can use the Epley formula, which is commonly used for this purpose:
\[ 1RM = w \times (1 + \frac{r}{30}) \]
where \( w \) is the weight lifted and \( r \) is the number of repetitions performed.

Here's a Python function to calculate the 1RM:

```python
def calculate_one_rep_max(weight: float, reps: int) -> float:
    """
    Calculate the one-repetition maximum (1RM) using the Epley formula.
    
    Parameters:
    weight (float): The weight lifted (in kilograms or pounds).
    reps (int): The number of repetitions performed.
    
    Returns:
    float: The estimated one-repetition maximum (1RM).
    """
    if weight <= 0 or reps <= 0:
        raise ValueError("Weight and repetitions must be positive values")
    one_rep_max = weight * (1 + reps / 3